# 用Cython加速Python程序的一个简单例子

Python是我们非常喜欢的一种语言，我们的LOGO中的“L”就是一条代表了Python的蛇 :)

然而，由于Python本身的语言特性，使得它在特定的情况下会出现速度瓶颈，本文就使用Cython做一个简单的例子，说明如何使用C和Python的混合（这大概并不是精确意义下的混合编程）达到优化Python程序的目的。

### 几点说明：
- 本文大量参考了参考文献[1]
- 本文的写作环境使用Python 2.7, IPython Notebook 1.1.0, Cython 0.17.2和Windows 7 64bit
- 为了展示方便，本文并没有描述如何在实际开发环境中编译和使用Cython模块，请参考官方文档（http://docs.cython.org/src/tutorial/cython_tutorial.html）
- Cython、Numba、PyPy等都可以用于优化Python程序的运行效率，numpy等专门进行科学计算的模块本身也会有优化，请不要把自己的思路限制在这里
- Cython实际上是一种新的编程语言，主要用于开发Python扩展模块，我们基于Cython所做的优化，本质是使用了Cython这种编程语言编写了融合C和Python风格的程序，然后通过编译称为程序模块，如其官方文档所说：[Cython] is a programming language that makes writing C extensions for the Python language as easy as Python itself. It aims to become a superset of the [Python] language which gives it high-level, object-oriented, functional, and dynamic programming. Its main feature on top of these is support for optional static type declarations as part of the language. The source code gets translated into optimized C/C++ code and compiled as Python extension modules. This allows for both very fast program execution and tight integration with external C libraries, while keeping up the high programmer productivity for which the Python language is well known. [2]
- 关于Cython在ipn中的使用，请参见：http://docs.cython.org/src/quickstart/build.html

#### 首先，为ipn设置ipython环境

In [190]:
%load_ext cythonmagic

#### 以下是我们的测试函数，这个函数来自参考文献[1]，用于计算地球表面两点之间的距离

In [169]:
import math

def great_circle_1(lon1,lat1,lon2,lat2):
    radius = 3956 #miles
    x = math.pi/180.0

    a = (90.0-lat1)*(x)
    b = (90.0-lat2)*(x)
    theta = (lon2-lon1)*(x)
    c = math.acos((math.cos(a)*math.cos(b))+(math.sin(a)*math.sin(b)*math.cos(theta)))

    return radius*c

In [170]:
%%time
lon1, lat1, lon2, lat2 = -72.345, 34.323, -61.823, 54.826
for i in xrange(500000):
    great_circle_1(lon1,lat1,lon2,lat2)

Wall time: 1.16 s


#### 经过测试，得到了运行时间的baseline。

在这个函数的运行过程中，数学运算占据了很大比例，Cython使我们指定变量的类型为C语言的数据类型，这也是最基本的加速手段，改动后的程序如下：

In [183]:
%%cython
import math

def great_circle_2(float lon1,float lat1,float lon2,float lat2):
    cdef float radius = 3956.0
    cdef float pi = 3.14159265
    cdef float x = pi/180.0
    cdef float a,b,theta,c

    a = (90.0-lat1)*(x)
    b = (90.0-lat2)*(x)
    theta = (lon2-lon1)*(x)
    c = math.acos((math.cos(a)*math.cos(b)) + (math.sin(a)*math.sin(b)*math.cos(theta)))
    return radius*c

In [184]:
%%time
lon1, lat1, lon2, lat2 = -72.345, 34.323, -61.823, 54.826
for i in xrange(500000):
    great_circle_2(lon1,lat1,lon2,lat2)

Wall time: 1.02 s


####可以看到，速度得到了一定程度的提升

这基本上可以说是最为无脑的“优化”方法。在此基础上，如果我们继续分析可以发现，性能瓶颈可能存在于Python的Math模块，Cython允许我们引入C语言的函数库，这时Cython加速的重要途径，改动后的程序如下（顺便可以把函数的定义也改成C风格的）：

In [187]:
%%cython
cdef extern from "math.h":
    float cosf(float theta)
    float sinf(float theta)
    float acosf(float theta)

cdef great_circle_3(float lon1,float lat1,float lon2,float lat2):
    cdef float radius = 3956.0
    cdef float pi = 3.14159265
    cdef float x = pi/180.0
    cdef float a,b,theta,c

    a = (90.0-lat1)*(x)
    b = (90.0-lat2)*(x)
    theta = (lon2-lon1)*(x)
    c = acosf((cosf(a)*cosf(b)) + (sinf(a)*sinf(b)*cosf(theta)))

    return radius*c

In [188]:
%%time
lon1, lat1, lon2, lat2 = -72.345, 34.323, -61.823, 54.826
for i in xrange(500000):
    great_circle_3(lon1,lat1,lon2,lat2)

Wall time: 272 ms


####经过测试，速度有了非常明显的提升。

其实，这段程序依然有优化的空间，其中之一是使用C语言的循环代替Python的for循环，因为使用一个Python的for循环执行一个C风格的函数，会存在一些转换开销，不过，这里就不做演示咯（懒！）

## Reference:
[1] http://blog.perrygeo.net/2008/04/19/a-quick-cython-introduction/

[2] http://docs.cython.org/src/quickstart/overview.html